Esta guia está basada en los tutoriales y videos del profesor Glen Evenbly en www.tensors.net, charlas del profesor Miles Stoudenmire http://scgp.stonybrook.edu/video_portal/video.php?id=3490, en http://tensornetwork.org/ y documentaciones de la libreria tensornetwork

# Terminologia tensorial



*   ***Rango:*** Numero de indices que tiene.
*   ***Dimension:*** Tamaño del indice de un tensor, es decir, numero maximo de valores que un indice puede tener



Para nuestros propositos, los tensores seran simplemente arreglos multidimensionales de numeros reales o complejos


In [0]:
!pip install tensornetwork
!pip install tensorflow

     |████████████████████████████████| 122kB 4.8MB/s 
     |████████████████████████████████| 71kB 26.4MB/s 
  Created wheel for opt-einsum: filename=opt_einsum-3.0.1-cp36-none-any.whl size=58500 sha256=7914721acd5d86de7b532d09289f19971d3a1cef355f06948e5dc39ca0c862c5
  Stored in directory: /root/.cache/pip/wheels/91/98/8d/10e3d4e04c959597a411b91acd3695e9e2d210e68ce3427aad
Successfully built opt-einsum
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
#Inicializar algunos tensores con numpy y tensornetwork

import numpy as np
import tensornetwork
import tensorflow as tf
net = tensornetwork.TensorNetwork() 

#Crear un tensor con entradas generadas aleatoriamente, de orden 3, dimensiones: 2,3,4

A = np.random.rand(2,3,4)
a = net.add_node(tf.random_normal((2,3,4))) # Tambien se pueden pasar arreglos de numpy.

In [0]:
#Veamos la forma del tensor A
print(A)

[[[0.94111175 0.37097395 0.34396331 0.09643969]
  [0.09865654 0.99171963 0.93498012 0.37523245]
  [0.93882088 0.94935283 0.70436233 0.59749847]]

 [[0.30347484 0.4512888  0.02763459 0.59168492]
  [0.04948909 0.20691906 0.52206017 0.7753557 ]
  [0.2099888  0.9395721  0.37897086 0.19353244]]]


In [0]:
#Veamos la forma del tensor a
print(a)

__Node_1


In [0]:
type(a)

tensornetwork.network_components.Node

Para utilizar este nodo como tensor simplemente recurrimos al metodo .tensor:

In [0]:
print(a.tensor)

Tensor("random_normal:0", shape=(2, 3, 4), dtype=float32)


In [0]:
#Ahora, como en tensorflow, para ejecutar el codigo se hace a partir de una sesión

with tf.Session() as sess:
  at = sess.run(a.tensor)

print(at)

[[[ 1.4246813   0.8147423   0.18641667 -0.46471953]
  [ 0.03354518 -1.2724595   0.04596779  0.13286783]
  [ 1.3711728   2.4674466   0.41749954 -0.9326194 ]]

 [[-0.69581693  0.583989   -0.5381667   0.4300234 ]
  [-0.21403661 -1.7804728   0.20250194 -1.3256449 ]
  [ 0.66473156 -0.4647254   1.4172493   0.40169504]]]


#Notacion Diagramatica

*  Cada tensor será representado por un circulo con un numero de lineas que corresponden a su rango.

* Cada linea estará etiquetada con una letra necesaria para relacionar la ecuacion con el diagram.

![texto alternativo](https://static.wixstatic.com/media/d91e93_6a91aeea94304044a62cc32d064aaf61~mv2.png/v1/fill/w_660,h_340,al_c,q_90/d91e93_6a91aeea94304044a62cc32d064aaf61~mv2.webp)


In [0]:
# En TensorNetwork las lineas libres son creadas automaticamente con la creacion del nodo.
# podemos acceder a ellas de la siguiente manera.
edgei = a.get_edge(0)
edgej = a.get_edge(1)
edgek = a.get_edge(2)

#Las dimensiones de cada linea son
print('Dimensiones de las lineas')
print(edgei.dimension)
print(edgej.dimension)
print(edgek.dimension)

print('Rango del tensor')
print(a.get_rank())

Dimensiones de las lineas
2
3
4
Rango del tensor
3


# Contraccion

Un indice compartido por dos tensores indica una contraccion (o sumatoria) sobre éste indice.

![texto alternativo](https://static.wixstatic.com/media/d91e93_4a2852f173884f0c96aa869aa6524ee7~mv2.png/v1/fill/w_954,h_325,al_c,q_90/d91e93_4a2852f173884f0c96aa869aa6524ee7~mv2.webp)

# Operaciones de permutar y remodelar (reshape)

* ***Permutar:*** cambia el orden del índice de un tensor (pero no cambia el número de índices)

* ***remodelar:*** combina una colección de índices en un índice más grande (o viceversa), puede cambiar el número de índices pero no la dimensión total.

![texto alternativo](https://static.wixstatic.com/media/d91e93_6afe662a2ff44b3c8df018f0ac787b09~mv2.png/v1/fill/w_950,h_368,al_c,q_90/D18.webp)

Nota:

*    Hay que permutar los nodos para afirmar que los ejes de su resultado están en el orden correcto

*    Contraer sin reordenar puede producir calculos intermedios, lo que se traduce en mas costo computacional

* Permutar reordena el almacenamiento de los elementos de un tensor en la memoria de la computadora, por lo que incurre en un costo computacional (a menudo no despreciable). Por el contrario, la función de remodelación deja los elementos de un tensor sin cambios en la memoria, solo cambia los metadatos de cómo debe interpretarse el tensor (y por lo tanto incurre en un costo insignificante).

<img src="https://media.arxiv-vanity.com/render-output/1119431/bad-contract.png" width="400">

In [0]:
#Permutar y remodelar en numpy

#Permutar el tensor A de orden 4 con dimensiones 1,2,3,4

A = np.random.rand(1,2,3,4)

Atilda = A.transpose(3,0,1,2)

print(Atilda.shape) #Debe imprimir (4,1,2,3)

#Remodelar el tensor B de orden 3 con dimensiones 4,4,4

B = np.random.rand(4,4,4)

Btilda = B.reshape(4,4*4)

print(Btilda.shape) #Debe imprimir (4,16)

(4, 1, 2, 3)
(4, 16)


In [0]:
#Permutar y remodelar en tensornetwork

#Permutar el tensor D de orden 3 con dimensiones 1,2,3

D = net.add_node(np.zeros((1, 2, 3)))
d1 = D[0]
d2 = D[1]
d3 = D[2]
D.reorder_edges([d3, d1, d2])
# Si conoce los valores de los ejes, puede hacer equivalentemente
# a.reorder_axes([2, 0, 1])
print(D.tensor.shape) # Debe imprimir (3, 1, 2)

#Remodelar el tensor E de orden 3 con dimensiones 2,3,4

E = net.add_node(np.ones((2, 3, 4)))
e1 = E[0]
e2 = E[1]
e3 = E[2]
flattened_edge = net.flatten_edges([E[0], E[1]])

E.reorder_edges([flattened_edge,e3])

print(E.tensor.shape) #Debe imprimir (6,4)

(3, 1, 2)
(6, 4)


# Contraccion de tensor binario

Permutar y remodelar permiten que una contracción entre un par de tensores (que llamamos una contracción de tensor binario) se pueda escribir como una multiplicación matricial. 

![texto alternativo](https://static.wixstatic.com/media/d91e93_a62d3c7786494eada0bb5a2f3518d662~mv2.png/v1/fill/w_864,h_195,al_c,q_90/d91e93_a62d3c7786494eada0bb5a2f3518d662~mv2.webp)

Metodologia:

1.   Permutar los tensores A y B tal que los indices a contraer esten contiguos entre ambos tensores
2.   Remodele los tensores (entre los índices que se contraerán frente a los índices restantes)
3.   Hacer la multiplicación de matrices
4.   Remodele a C en un tensor, realice la permutación final si el orden del índice deseado es diferente del orden actual

![texto alternativo](https://static.wixstatic.com/media/d91e93_8c99c11e840e40e49bceaf9751df3158~mv2.png/v1/fill/w_555,h_490,al_c,q_90/d91e93_8c99c11e840e40e49bceaf9751df3158~mv2.webp)

In [0]:
##### Contraccion de tensor binario en numpy
i = 1; m = 2; j = 3; n = 4; k = 5; l = 6

#Cijkl = Aimjn*Bmkln

A = np.random.rand(i,m,j,n)  #Aimjn
B = np.random.rand(m,k,l,n)  #Bmkln

print('dimension A: ',A.shape) 
print('dimension B: ',B.shape)

#1) Permutar indices
Ap  = A.transpose(0,2,1,3)  #Aijmn 
Bp = B.transpose(0,3,1,2)   #Bmnkl

print('dimension Ap: ',Ap.shape)
print('dimension Bp: ',Bp.shape)

#2) Remodelar indices
App = Ap.reshape(i*j,m*n)   #A{ij}{mn}
Bpp = Bp.reshape(m*n,k*l)   #B{mn}{kl}

print('dimension App: ',App.shape)
print('dimension Bpp: ',Bpp.shape)

#3) Multiplicacion matricial
Cpp = np.matmul(App, Bpp)   #A{ij}{mn}B{mn}{kl}   

print('dimension Cpp: ',Cpp.shape)

#4) Reordenar tensor final
C   = Cpp.reshape(i,j,k,l)  #Cijkl

print('dimension C: ',C.shape)

dimension A:  (1, 2, 3, 4)
dimension B:  (2, 5, 6, 4)
dimension Ap:  (1, 3, 2, 4)
dimension Bp:  (2, 4, 5, 6)
dimension App:  (3, 8)
dimension Bpp:  (8, 30)
dimension Cpp:  (3, 30)
dimension C:  (1, 3, 5, 6)


In [0]:
#### Contraccion de tensor binario en tensorflow

i = 1; m = 2; j = 3; n = 4; k = 5; l = 6

A = net.add_node(np.random.rand(i,m,j,n))  
B = net.add_node(np.random.rand(m,k,l,n))

print('dimension A: ',A.tensor.shape)
print('dimension B: ',B.tensor.shape)

#1) Permutar indices

a0 = A[0];                         b0 = B[0]
a1 = A[1];                         b1 = B[1]
a2 = A[2];                         b2 = B[2]
a3 = A[3];                         b3 = B[3]
A.reorder_edges([a0, a2, a1,a3]);  B.reorder_edges([b0, b3, b1, b2])

print('dimension Ap: ',A.tensor.shape)
print('dimension Bp: ',B.tensor.shape)

#2) Remodelar indices

beta1_edge = net.flatten_edges([a1,a3])
beta2_edge = net.flatten_edges([b0,b3])

A.reorder_edges([a0,a2,beta1_edge])
B.reorder_edges([beta2_edge,b1,b2])

print('dimension App: ',A.tensor.shape)
print('dimension Bpp: ',B.tensor.shape)

#3) Multiplicacion matricial
edge = net.connect(A[2], B[0])
C = net.contract(edge)
print('dimension C: ',C.tensor.shape)

dimension A:  (1, 2, 3, 4)
dimension B:  (2, 5, 6, 4)
dimension Ap:  (1, 3, 2, 4)
dimension Bp:  (2, 4, 5, 6)
dimension App:  (1, 3, 8)
dimension Bpp:  (8, 5, 6)
dimension C:  (1, 3, 5, 6)


# Costos de contraccion

Formas de calcular el costo de la contraccion tensorial:

![texto alternativo](https://static.wixstatic.com/media/d91e93_a03f6f99ee3f430ca094f6c7514870c3~mv2.png/v1/fill/w_948,h_201,al_c,q_90/D19.webp)

Hay dos formas para contraer una red con mas de dos tensores:



1.   En un solo paso como la suma directa de los indices internos de la red
2.   Una secuencia de N-1 contracciones binarias. 

![texto alternativo](https://static.wixstatic.com/media/d91e93_27f38171fdb04152a27551492a6c6b7d~mv2.png/v1/fill/w_750,h_83,al_c,q_90/d91e93_27f38171fdb04152a27551492a6c6b7d~mv2.webp)

![texto alternativo](https://static.wixstatic.com/media/d91e93_5ca5f98d307b404f8be574e105910826~mv2.png/v1/fill/w_750,h_250,al_c,q_90/d91e93_5ca5f98d307b404f8be574e105910826~mv2.webp)



In [0]:
##### Evaluar Red tensorial -A-B-C-

import time
start1 = time.time()

d = 20
A = np.random.rand(d,d) 
B = np.random.rand(d,d)
C = np.random.rand(d,d)
# Via sumatoria de indices internos
F0 = np.zeros((d,d))
for di in range(d):
    for dj in range(d):
        for dk in range(d):
            for dl in range(d):
                F0[di,dj] = F0[di,dj] + A[di,dk]*B[dk,dl]*C[dl,dj]
                
end1 = time.time()
print(end1 - start1)
            
start2 = time.time()
# Via secuencia de contracciones binarias
F1 = (A @ B) @ C 

end2 = time.time()
print(end2 - start2)

0.24427270889282227
0.00014829635620117188


# Contraccion de redes tensoriales

Formas de contraer una red tensorial eficientemente:

1.   Determinar la secuencia optima (Minimizan numero de multiplicaciones escalares o tamaño de tensores intermediarios usados en la contraccion) de N-1 contracciones binarias
2.   Evalue cada contraccion binaria como una multiplicacion matricial tomando una apropiada permutacion y remodelacion tensorial

<img src="https://static.wixstatic.com/media/d91e93_0aec134314bc4a86b3fd10913c120f2f~mv2.png/v1/fill/w_644,h_525,al_c,q_90/d91e93_0aec134314bc4a86b3fd10913c120f2f~mv2.webp" width="400">



# ncon (Network contractor)

Disminuye el esfuerzo requerido para implementar una contracción. Esta función realiza automáticamente una secuencia deseada de permutaciones, remodelaciones y multiplicaciones de matrices necesarias para evaluar una red tensora. El primer paso para usar 'ncon' para evaluar una red es hacer un diagrama etiquetado de la red de manera que:

*   Cada índice interno está etiquetado con un entero positivo único (típicamente enteros secuenciales que comienzan desde 1).

*   Los índices externos del diagrama (si hay alguno) están etiquetados con enteros negativos secuenciales [-1, -2, -3, ...] que denotan el orden de índice deseado en el tensor final (con -1 como primer índice, -2 como el segundo, etc.)

<img src="https://static.wixstatic.com/media/d91e93_423fc7cd9dba43ad859ddc3a3b16b772~mv2.png/v1/fill/w_949,h_295,al_c,q_90/d91e93_423fc7cd9dba43ad859ddc3a3b16b772~mv2.webp" width="600"> 

<img src="https://static.wixstatic.com/media/d91e93_eb675db6d96a47ce91a8ddbf265b2605~mv2.png/v1/fill/w_259,h_270,al_c,q_90/d91e93_eb675db6d96a47ce91a8ddbf265b2605~mv2.webp" width="150">

Después de esto, la rutina 'ncon' se llama de la siguiente manera,

**OutputTensor = ncon(TensorArray, IndexArray, con_order)**,

Con argumentos:

*    **TensorArray:** matriz de celdas 1D que contiene los tensores que comprenden la red

*    **IndexArray:** Matriz de vectores de celdas 1D, donde el elemento kth es un vector de las etiquetas enteras del diagrama en el tensor kth de "TensorArray" (ordenado siguiendo el orden de índice correspondiente en este tensor).

*    **con_order:** un vector que contiene las etiquetas enteras positivas del diagrama, que se usa para especificar el orden en el que "ncon" contrae los índices. Tenga en cuenta que "con_order" es una entrada opcional que se puede omitir si lo desea, en cuyo caso "ncon" se contraerá en orden ascendente de las etiquetas de índice.



In [0]:
from tensornetwork import ncon
sess = tf.InteractiveSession()

In [0]:
##### Contraccion usando ncon

#Ejemplo sencillo linea tensores -A-B-

a = tf.random_normal((2,2))
b = tf.random_normal((2,2))
c = ncon([a,b], [(-1,1),(1,-2)])
print(tf.norm(tf.matmul(a,b) - c).eval()) # imprime cero

0.0


In [0]:
#Ejemplo que se muestra en las figuras de arriba 

d = 10
A = tf.random_normal((d,d,d)); B = tf.random_normal((d,d,d,d))
C = tf.random_normal((d,d,d)); D = tf.random_normal((d,d))

#Tensores involucrados en la red
TensorArray = [A,B,C,D]

#etiquetado de indices
IndexArray = [[1,-2,2],[-1,1,3,4],[5,3,2],[4,5]]

#Uso de funcion ncon
E = ncon(TensorArray,IndexArray,con_order = [5,3,4,1,2])

print(E.shape)

(10, 10)
